In [1]:
import pandas as pd 


df = pd.read_csv("data/uber_data.csv")
df.columns




In [10]:
#Changing data columns into date format 
df['trip_id'] = df.index
df['tpep_pickup_datetime']=pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime']=pd.to_datetime(df['tpep_dropoff_datetime'])
#Extract weekday,month,year,hour needed in our datetime_dimension for pick and drop date 
datetime_dim = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].drop_duplicates().reset_index(drop=True)
datetime_dim['tpep_pickup_datetime'] = datetime_dim['tpep_pickup_datetime']
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday
datetime_dim['tpep_dropoff_datetime'] = datetime_dim['tpep_dropoff_datetime']
datetime_dim['drop_hour'] = datetime_dim['tpep_dropoff_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_dropoff_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_dropoff_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_dropoff_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_dropoff_datetime'].dt.weekday
datetime_dim['datetime_id']=datetime_dim.index 
#passenger_count_dim 
passenger_count_dim=df[["passenger_count"]].drop_duplicates().reset_index(drop=True)
passenger_count_dim["passenger_count_id"]=passenger_count_dim.index
passenger_count_dim=passenger_count_dim[['passenger_count_id','passenger_count']]
#trip_distance_dim 
trip_distance_dim=df[["trip_distance"]].drop_duplicates().reset_index(drop=True)
trip_distance_dim["trip_distance_id"]=trip_distance_dim.index 
trip_distance_dim = trip_distance_dim[["trip_distance_id","trip_distance"]]
#rate_code_dim
rate_code_type= {1:"Standard rate",2:"JFK",3:"Newark",4:"Nassau or Westchester",5:"Negotiated fare", 6:"Group ride"}
rate_code_dim = df[["RatecodeID"]].drop_duplicates().reset_index(drop=True) 
rate_code_dim["rate_code_name"]=rate_code_dim["RatecodeID"].map(rate_code_type)
rate_code_dim["rate_code_id"]=rate_code_dim.index
rate_code_dim=rate_code_dim[["rate_code_id","RatecodeID","rate_code_name"]]
rate_code_dim
#payment_type_dim 
payment_type_name={1:"Credit Card",2:"Cash",3:"No charge",4:"Dispute",5:"Unknown",6:"Voided trip"}
payment_type_dim=df[["payment_type"]].drop_duplicates().reset_index(drop=True)
payment_type_dim["payment_type_name"]=payment_type_dim["payment_type"].map(payment_type_name)
payment_type_dim["payment_type_id"]=payment_type_dim.index
payment_type_dim=payment_type_dim[["payment_type_id","payment_type","payment_type_name"]]
#pickup_location_dim
pickup_location_dim=df[["pickup_latitude","pickup_longitude"]].drop_duplicates().reset_index(drop=True)
pickup_location_dim["pickup_location_id"]=pickup_location_dim.index
pickup_location_dim=pickup_location_dim[["pickup_location_id","pickup_latitude","pickup_longitude"]]
#dropoff_location_dim 
dropoff_location_dim=df[["dropoff_latitude","dropoff_longitude"]].drop_duplicates().reset_index(drop=True)
dropoff_location_dim["dropoff_location_id"]=dropoff_location_dim.index
dropoff_location_dim=dropoff_location_dim[["dropoff_location_id","dropoff_latitude","dropoff_longitude"]]




In [11]:
fact_table=df.merge(passenger_count_dim, on ="passenger_count")\
             .merge(trip_distance_dim,on="trip_distance")\
             .merge(rate_code_dim,on="RatecodeID")\
             .merge(payment_type_dim,on="payment_type")\
             .merge(pickup_location_dim,on=["pickup_latitude","pickup_longitude"])\
             .merge(dropoff_location_dim,on=["dropoff_latitude","dropoff_longitude"])\
             .merge(datetime_dim,on=["tpep_pickup_datetime","tpep_dropoff_datetime"])[["trip_id","VendorID","datetime_id","passenger_count_id","trip_distance_id"\
                                                                                       ,"rate_code_id","store_and_fwd_flag","pickup_location_id","dropoff_location_id"\
                                                                                       ,"payment_type_id","fare_amount","extra","mta_tax","tip_amount","tolls_amount",\
                                                                                       "improvement_surcharge","total_amount"]]




In [12]:
fact_table.head()

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,N,0,0,0,9.0,0.5,0.5,2.05,0.0,0.3,12.35
1,1496,2,1491,0,0,0,N,1481,1484,0,10.5,0.0,0.5,2.26,0.0,0.3,13.56
2,2840,2,2834,0,0,0,N,2816,2819,0,9.5,0.0,0.5,1.25,0.0,0.3,11.55
3,3495,2,3488,0,0,0,N,3465,3470,0,13.5,0.0,0.5,2.00,0.0,0.3,16.30
4,3930,2,3923,0,0,0,N,3899,3903,0,10.5,0.0,0.5,2.26,0.0,0.3,13.56


In [3]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,trip_id
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35,0
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35,1
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80,2
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62,3
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80,4
